In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pj2-project").getOrCreate()

# https://sparkbyexamples.com/apache-hive/pyspark-save-dataframe-to-hive-table/
SparkSession.builder.enableHiveSupport()

In [6]:
import pyspark.sql.functions  as F

In [3]:
def clean_null(df):
    
    df_cleaned = df
    for col in df.columns:
        df_cleaned = df_cleaned.filter(F.col(col).isNotNull())
        df_cleaned = df_cleaned.filter(F.col(col) != 'None')
        df_cleaned = df_cleaned.filter(F.col(col) != 'null')
    return df_cleaned

# first process the business dataset

In [3]:
tbl_business = spark.read.option("header",True).json("/user/pytrick/yelp_data/yelp_academic_dataset_business.json")

23/03/12 19:09:31 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:

# tbl_business.count() - tbl_business.dropDuplicates().count()

+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|address|attributes|business_id|categories|city|hours|is_open|latitude|longitude|name|postal_code|review_count|stars|state|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|      0|     13744|          0|       103|   0|23223|      0|       0|        0|   0|          0|           0|    0|    0|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+



In [13]:
tbl_business.show(10,True)

+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|34.4266787|-119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1| 38.551126|  -90.335695

In [17]:
tbl_business.count(), tbl_business.distinct().count()

(150346, 150346)

In [15]:
tbl_business.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state']

In [21]:
tbl_business_cleaned = tbl_business
for i in tbl_business.columns:
    tbl_business_cleaned = tbl_business_cleaned.filter(F.col(i).isNotNull())

In [22]:
tbl_business_cleaned.count()

117618

In [26]:
tbl_business_cleaned.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in tbl_business.columns]).show()

+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|address|attributes|business_id|categories|city|hours|is_open|latitude|longitude|name|postal_code|review_count|stars|state|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|      0|         0|          0|         0|   0|    0|      0|       0|        0|   0|          0|           0|    0|    0|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+



In [70]:
# save to hive
tbl_business_cleaned.write.mode('overwrite') \
    .saveAsTable("yelp.business")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/03/12 19:47:42 WARN org.apache.hadoop.hive.ql.session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


# 2.process the checking dataset

In [1]:
tbl_checkin = spark.read.option("header",True).json("/user/pytrick/yelp_data/yelp_academic_dataset_checkin.json")

In [7]:
tbl_checkin_cleaned = clean_null(tbl_checkin)

In [8]:
tbl_checkin_cleaned.show(10,True)

+--------------------+--------------------+
|         business_id|                date|
+--------------------+--------------------+
|---kPU91CF4Lq2-Wl...|2020-03-13 21:10:...|
|--0iUa4sNDFiZFrAd...|2010-09-13 21:43:...|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:...|
|--7PUidqRWpRSpXeb...|2011-02-15 17:12:...|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:...|
|--8IbOsAAxjKRoYsB...|2015-06-06 01:03:...|
|--9osgUCSDUWUkoTL...|2015-06-13 02:00:...|
|--ARBQr1WMsTWiwOK...|2014-12-12 00:44:...|
|--FWWsIwxRwuw9vIM...|2010-09-11 16:28:...|
|--FcbSxK1AoEtEAxO...|2017-08-18 19:43:...|
+--------------------+--------------------+
only showing top 10 rows



In [59]:
tbl_checkin_cleaned.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_c: timestamp (nullable = true)



In [63]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [66]:
tbl_checkin_cleaned = tbl_checkin_cleaned.withColumn("datetime",F.to_timestamp(F.col('date'),'yyyy-MM-dd HH:mm:ss'))

In [67]:
tbl_checkin_cleaned.show(10,True)

+--------------------+--------------------+-------------------+-------------------+
|         business_id|                date|             date_c|           datetime|
+--------------------+--------------------+-------------------+-------------------+
|---kPU91CF4Lq2-Wl...|2020-03-13 21:10:...|2020-03-13 21:10:56|2020-03-13 21:10:56|
|--0iUa4sNDFiZFrAd...|2010-09-13 21:43:...|2010-09-13 21:43:09|2010-09-13 21:43:09|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:...|2013-06-14 23:29:17|2013-06-14 23:29:17|
|--7PUidqRWpRSpXeb...|2011-02-15 17:12:...|2011-02-15 17:12:00|2011-02-15 17:12:00|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:...|2014-04-21 20:42:11|2014-04-21 20:42:11|
|--8IbOsAAxjKRoYsB...|2015-06-06 01:03:...|2015-06-06 01:03:19|2015-06-06 01:03:19|
|--9osgUCSDUWUkoTL...|2015-06-13 02:00:...|2015-06-13 02:00:57|2015-06-13 02:00:57|
|--ARBQr1WMsTWiwOK...|2014-12-12 00:44:...|2014-12-12 00:44:23|2014-12-12 00:44:23|
|--FWWsIwxRwuw9vIM...|2010-09-11 16:28:...|2010-09-11 16:28:39|2010-09-11 16

In [72]:
# write to hive
tbl_checkin_cleaned.select("business_id",'datetime','date').write.mode('overwrite') \
    .saveAsTable("yelp.checkin")

In [74]:
df = spark.read.table("yelp.checkin")
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- date: string (nullable = true)



# 3.process the review dataset, this table contains the checking time of the customer for specific business_id

In [76]:
tbl_review = spark.read.option("header",True).json("/user/pytrick/yelp_data/yelp_academic_dataset_review.json")

In [77]:
tbl_review_cleaned  = clean_null(tbl_review)

In [78]:
tbl_review_cleaned.count(), tbl_review.count()

(6990280, 6990280)

In [79]:
tbl_review_cleaned.show(10)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [81]:
tbl_review_cleaned.write.mode('overwrite') \
    .saveAsTable("yelp.review")

# 4 Tip

In [83]:
tbl_tip = spark.read.option("header",True).json("/user/pytrick/yelp_data/yelp_academic_dataset_tip.json")


In [84]:
tbl_tip_cleaned = clean_null(tbl_tip)

In [85]:
tbl_tip_cleaned.write.mode('overwrite') \
    .saveAsTable("yelp.tip")

# 5 user 

In [87]:
tbl_user = spark.read.option("header",True).json("/user/pytrick/yelp_data/yelp_academic_dataset_user.json")

In [88]:
tbl_user_cleaned = clean_null(tbl_user)

In [91]:
tbl_user_cleaned.write.mode('overwrite') \
    .saveAsTable("yelp.user")

In [92]:
df = spark.read.table("yelp.user")
df.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [93]:
df.show(10)

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+---------------+----+--------------------+-----+--------------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer|cool|          elite|fans|             friends|funny|          name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+---------------+----+--------------------+-----+--------------+------------+------+--------------------+-------------------+
|         4.7